# Looking Over Some Basic Functionality

## Loading various modules
Currently **collatzpy** is broken up into three basic modules,
**fpaths**, **tree**, and **plot**

### fpaths
fpaths is a single function that generates some directories for basic things you might create,
a root path, images/graphs, pickles/dills/data, and dot files.
It is entirely optional, you can always specify your own paths each call you make to a function
that requires it.  Its implementation is frankly silly but still convenient when you're just
playing around and generating lots of data.

Without arguments, fpaths will generate a root folder in,
    *home/youruser/Documents/collatzpy*
or whatever is equivalent for your OS.

Passing *dir* with your prefered path will create those folders there instead.

**fpaths stores these locations in a nice session.json file!**

Future calls to **fpaths**, without arguments will return those paths to you so you may continue
as before.

Passing *reset=True* will reset the paths to the default specified above.  Alternatively,
you can pass a new *dir* and it will update accordingly.

*NOTE*

**fpaths** will not delete previous directories or their contents, you need to do thatyourself.

In [1]:
import collatzpy.fpaths as cfpaths # generates a collection of convenient paths for file saving...

paths = cfpaths()
rpath = paths['root']
ipath = paths['imgs']
ppath = paths['pickles']
dpath = paths['dots']

## tree
Tree has two associated classes, ***CollatzNode*** and ***CollatzTree***, and the functions, *save_tree*, and *load_tree*.

***CollatzNode*** <br>
The node class is trivial but currently contains 3 noteworthy pieces of data, *n*, *next*, and *seq_len*. <br>
* *n* | The value of the node. <br>
* *seq_len* | The calculated sequence length it takes to get to 1. <br>
* *next* | A pointer to the next value in its sequence. <br>

***CollatzTree*** <br>
The meat and potatoes.<br>
The tree class stores the nodes, node sequences, and offers a set of very basic class functions to generate
sequences or return data to the user.

In brief, they are: <br>
* *has(n)* | Boolean if the tree contains the node and respective sequence for the integer 'n'.
* *splay(n)* | Returns a dict of the given node instance's variables *n*, *next*, and *seq_len*.
* *best()* | Takes no arguments, returns the same data as splay except from the node in the tree with the highest sequence length.
* *calc_next(n)* | Generally only used internally, will return the next integer in the sequence, given 'n'.
* *collect(n)* | Given an integer 'n', will gather all subsequent nodes in the sequence and add them to the tree. Collect will also update the best node if one is found, and will not waste time repeatedly collecting a number it has already sequenced.
* *collect_from_range(a, b)* | Will collect all numbers in the range of (a, b), inclusive.
* *collect_from_list(some_list)* | Will collect all the numbers in a given list.
* *path(n)* | Will return a list of the number sequence that goes from 'n' to 1. Will return an empty list for values of 'n' not previously calculated.
* *longest_seq()* | Takes no arguments.  A hold over and slower method of getting the collatz number with the current longest sequence.  Just use *best()*, as it stores that node directly.
* You can also call on the tree instance directly, i.e., mytree(n).  It will return the node itself, not just a copy of the data as with the splay function.

The CollatzTree is built as a collection of dict references and linked nodes.  It's very compact and will only store a number exactly once, taking advantage of the fact that all paths lead to 1.  Lookups are as fast as a dict and sequence traversal is done as a one-way linked list, moving from 'n' to 1.  Peformance is great even for reasonably large numbers,
but no efforts have been made to equal the size, scope, and performance of other programs.  This is more so for making interesting images and viewing smaller subsets of the collatz sequence. <br>
*For reference, as of now, the class can generate and store every sequence from 1 to 1 million in less than 3 seconds on the desktop tested.  That's compactifying a sequence 131,434,424 numbers in total length, to a dict of *just* 2,168,611.  And every node in the tree will know its path back to 1!*


In [7]:
import collatzpy.tree as ctree # import the tree module

mytree = ctree.CollatzTree() # create a tree
mytree.collect(101) # collect the path for a specific number
mytree.collect_from_range(2, 27) # collect the paths in a given range
my_list = [x for x in range(28, 98)]
mytree.collect_from_list(my_list) # collect the paths in a given list

In [3]:
path27 = mytree.path(27) # return a list of the path from 27 to 1
print(path27)

[27, 82, 41, 124, 62, 31, 94, 47, 142, 71, 214, 107, 322, 161, 484, 242, 121, 364, 182, 91, 274, 137, 412, 206, 103, 310, 155, 466, 233, 700, 350, 175, 526, 263, 790, 395, 1186, 593, 1780, 890, 445, 1336, 668, 334, 167, 502, 251, 754, 377, 1132, 566, 283, 850, 425, 1276, 638, 319, 958, 479, 1438, 719, 2158, 1079, 3238, 1619, 4858, 2429, 7288, 3644, 1822, 911, 2734, 1367, 4102, 2051, 6154, 3077, 9232, 4616, 2308, 1154, 577, 1732, 866, 433, 1300, 650, 325, 976, 488, 244, 122, 61, 184, 92, 46, 23, 70, 35, 106, 53, 160, 80, 40, 20, 10, 5, 16, 8, 4, 2, 1]


In [12]:
node = mytree(27) # retrieve the node object associated with the given number
print('Calling on the tree directly: ', node, '\n')
# calling directly on the tree for a nonexistent node will return None

node_info = mytree.splay(27) # retrieve the information about the given node
print('Calling mytree.splay(27): ', node_info, '\n')
# calling splay for a nonexistent node will return Nothing

best_node = mytree.best() # retrieve info associated with the node with the longest path sequence
print('Calling mytree.best(): ', best_node, '\n')

Calling on the tree directly:  <collatzpy.tree.collatz_node.CollatzNode object at 0x7f3d657ff890> 

Calling mytree.splay(27):  {'n': 27, 'seq_len': 111, 'next': 82} 

Calling mytree.best():  {'n': 97, 'seq_len': 118, 'next': 292} 



### Saving/Loading tree data
Trees are stored using the dill extension from pickle.  They'll retain their
full state between sessions! Pretty convenient.

In [5]:
ctree.save_tree(mytree, ppath, 'mytree.pickle') # That sweet fpaths var...

mytree_copy = ctree.load_tree(ppath, 'mytree.pickle')
